# Pandas Cheat Sheet

# Table of contents


- **The setup**: anaconda, Python, pandas, Jupyter
- **Importing data**: from csv (and options), from the web, creating from scratch, convering types, rename cols
- **Summarizing data**: len(df), shape, value_counts, head, tail, max(), min(), mean, dtype, info(), describe(), memory_usage(), scatter matrix, corr, isnull, notnull, unique(), nlargest
- **Selecting and computing**: select subset of row and cols, .loc, .iloc, drop columns, assign, apply/map/applymap, multiindex
- **Filtering and sorting**: >=, AND, OR, ==, ~, str.contains, str.startswith, sort_values, sort_index, filtering on sorted/unsorted, isin()
- **Split-apply-combine and pivots**: groupby, dt.month, dt.year, groupby.mean(), agg, stack, unstack, pivot, melt, merge
- **Time series manipulations**: downsampling, upsampling, rolling, mean, simple plotting
- **Plotting**: built-in plotting, advanced plotting, matplotlib, seaborn, styles, saving
- **Modeling and machine learning**: .value, feeding data, saving data
- **Misc tips and tricks**: pandas options, vectorization, timings with %%timeit, profiling with lprun

**principles:** small examples, no more than 5 rows. one or two data sets, no more.

# The setup

## Python and Anaconda

If you haven't done it, start by installing Python.
The [Anaconda Distribution](https://www.anaconda.com/download/) is great, install version `3.X`.
- If you're on Windows, you will get a program called *Anaconda Prompt*. Open in at run `conda --version` to verify that everything works.
- If you're on Linux, open a terminal and run `conda --version`.

## Pandas, NumPy and matplotlib

To install packages, run `conda install <package>`. The Anaconda distribution comes with the three packages we will require, namely [pandas](https://pandas.pydata.org/), [NumPy](http://www.numpy.org/) and [matplotlib](https://matplotlib.org/).

- **NumPy** implements $n$-dimensional arrays in Python for efficient computations. See the [arXiv](https://arxiv.org/pdf/1102.1523.pdf) paper for a nice introduction. To learn basic NumPy, consider doing these [100 NumPy exercises](https://github.com/rougier/numpy-100).
- **Matplotlib** is the most popular library for plotting in Python. See the beautiful [gallery](https://matplotlib.org/gallery.html) to get an overview of the capabilities of matplotlib.
- **Pandas** is a library for data analysis based on two objects, the [Series](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) and the [DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

## Jupyter

The [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/) is an environment in which you can run Python code, display graphs and work with data interactively. Think of it as a tool between the simple terminal and the full fledged IDE. Move to a directory using the `cd` command in the terminal, then run `jupyter notebook` to start up a notebook. 

## Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

In [2]:
for lib in [pd, np, matplotlib]:
    print(f'{lib.__name__.ljust(12)} version {lib.__version__}')

pandas       version 0.22.0
numpy        version 1.13.3
matplotlib   version 2.1.1


In [3]:
pd.set_option("display.max_rows", 2**6)
pd.set_option("display.max_columns", 2**6)

In [4]:
2**6

64

# Importing data

We'll use

In [5]:
!head data/movie_metadata.csv -n 2

In [6]:
df = pd.read_csv(r'data/movie_metadata.csv', sep = ',')
df.shape

(5043, 28)

# Summarizing data

In [7]:
df.shape

(5043, 28)

In [8]:
df.dtypes.head(10)

color                       object
director_name               object
num_critic_for_reviews     float64
duration                   float64
director_facebook_likes    float64
actor_3_facebook_likes     float64
actor_2_name                object
actor_1_facebook_likes     float64
gross                      float64
genres                      object
dtype: object

In [9]:
df.head(2).T

,0,1
color,Color,Color
director_name,James Cameron,Gore Verbinski
num_critic_for_reviews,723,302
duration,178,169
director_facebook_likes,0,563
actor_3_facebook_likes,855,1000
actor_2_name,Joel David Moore,Orlando Bloom
actor_1_facebook_likes,1000,40000
gross,7.60506e+08,3.09404e+08
genres,Action|Adventure|Fantasy|Sci-Fi,Action|Adventure|Fantasy


In [10]:
df.isnull().sum().sort_values()

cast_total_facebook_likes      0
imdb_score                     0
movie_imdb_link                0
num_voted_users                0
movie_title                    0
genres                         0
movie_facebook_likes           0
country                        5
actor_1_facebook_likes         7
actor_1_name                   7
language                      12
actor_2_facebook_likes        13
actor_2_name                  13
facenumber_in_poster          13
duration                      15
color                         19
num_user_for_reviews          21
actor_3_facebook_likes        23
actor_3_name                  23
num_critic_for_reviews        50
director_facebook_likes      104
director_name                104
title_year                   108
plot_keywords                153
content_rating               303
aspect_ratio                 329
budget                       492
gross                        884
dtype: int64

In [11]:
# len(df), shape, value_counts, head, tail, max(), min(), mean, dtype, info(), 
# describe(), memory_usage(), scatter matrix, corr, isnull, notnull, unique(), nlargest

# Selecting and computing

In [12]:
# - **Selecting and computing**: select subset of row and cols, .loc, .iloc, 
# drop columns, assign, apply/map/applymap, multiindex

In [13]:
columns = ['movie_title', 'director_name', 'country', 'content_rating', 'imdb_score']
df[columns].head(3)

,movie_title,director_name,country,content_rating,imdb_score
0,Avatar,James Cameron,USA,PG-13,7.9
1,Pirates of the Caribbean: At World's End,Gore Verbinski,USA,PG-13,7.1
2,Spectre,Sam Mendes,UK,PG-13,6.8


In [14]:
df.loc[100:102, columns]

,movie_title,director_name,country,content_rating,imdb_score
100,The Fast and the Furious,Rob Cohen,USA,PG-13,6.7
101,The Curious Case of Benjamin Button,David Fincher,USA,PG-13,7.8
102,X-Men: First Class,Matthew Vaughn,USA,PG-13,7.8


In [15]:
df_cols = df.loc[:, columns + ['director_facebook_likes', 'gross']]
df_cols.head(2)

,movie_title,director_name,country,content_rating,imdb_score,director_facebook_likes,gross
0,Avatar,James Cameron,USA,PG-13,7.9,0.0,760505847.0
1,Pirates of the Caribbean: At World's End,Gore Verbinski,USA,PG-13,7.1,563.0,309404152.0


In [16]:
df_cols = df_cols.drop(columns=['director_facebook_likes'])
df_cols.head(2)

,movie_title,director_name,country,content_rating,imdb_score,gross
0,Avatar,James Cameron,USA,PG-13,7.9,760505847.0
1,Pirates of the Caribbean: At World's End,Gore Verbinski,USA,PG-13,7.1,309404152.0


In [17]:
df_cols.nlargest(3, columns=['imdb_score'], keep='first')

,movie_title,director_name,country,content_rating,imdb_score,gross
2765,Towering Inferno,John Blanchard,Canada,NaN,9.5,NaN
1937,The Shawshank Redemption,Frank Darabont,USA,R,9.3,28341469.0
3466,The Godfather,Francis Ford Coppola,USA,R,9.2,134821952.0


In [18]:
df_cols = df_cols.assign(gross_log = lambda df: np.log(df.gross))

In [19]:
df_cols.describe(percentiles=[0.5]).applymap(lambda x: '{:,}'.format(round(x)))

,imdb_score,gross,gross_log
count,"5,043","4,159","4,159"
mean,6,"48,468,408",16
std,1,"68,452,990",2
min,2,162,5
50%,7,"25,517,500",17
max,10,"760,505,847",20


In [20]:
df_cols.corr()

,imdb_score,gross,gross_log
imdb_score,1.000000,0.198021,0.074280
gross,0.198021,1.000000,0.616034
gross_log,0.074280,0.616034,1.000000


# Filtering and sorting

In [21]:
df_cols[df_cols.country != 'USA'].head(3)

,movie_title,director_name,country,content_rating,imdb_score,gross,gross_log
2,Spectre,Sam Mendes,UK,PG-13,6.8,200074175.0,19.114199
4,Star Wars: Episode VII - The Force Awakens ...,Doug Walker,NaN,NaN,7.1,NaN,NaN
9,Harry Potter and the Half-Blood Prince,David Yates,UK,PG,7.5,301956980.0,19.525795


In [22]:
mask = ((df_cols.imdb_score > 8) & (df_cols.country != 'USA') & (df_cols.gross > 10**6))
df_cols[mask].nlargest(3, columns=['imdb_score'])

,movie_title,director_name,country,content_rating,imdb_score,gross,gross_log
4498,"The Good, the Bad and the Ugly",Sergio Leone,Italy,Approved,8.9,6100000.0,15.623799
270,The Lord of the Rings: The Fellowship of the R...,Peter Jackson,New Zealand,PG-13,8.8,313837577.0,19.564386
4029,City of God,Fernando Meirelles,Brazil,R,8.7,7563397.0,15.838831


In [23]:
# >=, AND, OR, ==, ~, str.contains, 
# str.startswith, sort_values, sort_index, filtering on sorted/unsorted, isin()

In [24]:
df_cols[df_cols.movie_title.str.lower().str.contains("ring")].nlargest(5, columns=['imdb_score'])

,movie_title,director_name,country,content_rating,imdb_score,gross,gross_log
2765,Towering Inferno,John Blanchard,Canada,NaN,9.5,NaN,NaN
339,The Lord of the Rings: The Return of the King,Peter Jackson,USA,PG-13,8.9,377019252.0,19.747807
270,The Lord of the Rings: The Fellowship of the R...,Peter Jackson,New Zealand,PG-13,8.8,313837577.0,19.564386
340,The Lord of the Rings: The Two Towers,Peter Jackson,USA,PG-13,8.7,340478898.0,19.645864
1196,The Conjuring 2,James Wan,USA,R,7.8,102310175.0,18.443520


In [25]:
df.country.value_counts().head(5)

USA        3807
UK          448
France      154
Canada      126
Germany      97
Name: country, dtype: int64

In [26]:
df.country.value_counts().sort_index().head(5)

Afghanistan     1
Argentina       4
Aruba           1
Australia      55
Bahamas         1
Name: country, dtype: int64

# Split-apply-combine and pivots

In [ ]:
- **Split-apply-combine and pivots**: groupby, dt.month, dt.year, groupby.mean(), agg, stack, unstack, pivot, melt, merge

## Directors with the most movies

In [36]:
(df_cols.groupby(df.director_name).nunique().movie_title.nlargest(5).to_frame())

,movie_title
director_name,
Steven Spielberg,26
Woody Allen,22
Clint Eastwood,20
Martin Scorsese,20
Ridley Scott,16


In [41]:
(df_cols.groupby(df.director_name).mean().imdb_score.nlargest(5).to_frame())

,imdb_score
director_name,
John Blanchard,9.5
Cary Bell,8.7
Mitchell Altieri,8.7
Sadyk Sher-Niyaz,8.7
Charles Chaplin,8.6


In [43]:
df_cols.head()

,movie_title,director_name,country,content_rating,imdb_score,gross,gross_log
0,Avatar,James Cameron,USA,PG-13,7.9,760505847.0,20.449494
1,Pirates of the Caribbean: At World's End,Gore Verbinski,USA,PG-13,7.1,309404152.0,19.550159
2,Spectre,Sam Mendes,UK,PG-13,6.8,200074175.0,19.114199
3,The Dark Knight Rises,Christopher Nolan,USA,PG-13,8.5,448130642.0,19.920595
4,Star Wars: Episode VII - The Force Awakens ...,Doug Walker,NaN,NaN,7.1,NaN,NaN


In [49]:
director_stats = (df_cols.groupby(df.director_name).agg({'imdb_score':np.mean, 'gross_log':np.sum, 'movie_title':'nunique'}))

director_stats.head(3)

,imdb_score,gross_log,movie_title
director_name,,,
A. Raven Cruz,1.9,0.000000,1
Aaron Hann,6.0,0.000000,1
Aaron Schneider,7.1,16.032162,1


In [69]:
director_stats_norm = ((director_stats - director_stats.mean()) / director_stats.std())
director_stats_norm.head(3)

,imdb_score,gross_log,movie_title
director_name,,,
A. Raven Cruz,-3.885663,-0.737455,-0.488133
Aaron Hann,-0.213965,-0.737455,-0.488133
Aaron Schneider,0.771125,-0.321155,-0.488133


In [71]:
director_stats_norm.assign(score = lambda df: df.sum(axis = 1)).nlargest(3, 'score')

,imdb_score,gross_log,movie_title,score
director_name,,,,
Steven Spielberg,1.112117,11.419216,11.609254,24.140587
Woody Allen,0.689712,7.299573,9.673672,17.662958
Clint Eastwood,0.883067,8.041961,8.705881,17.630909


In [82]:
(df.loc[:, ('title_year', 'content_rating', 'movie_title')]
 .groupby(['title_year', 'content_rating']).nunique().movie_title).unstack(1).fillna(0).tail(5)

content_rating,Approved,G,GP,M,NC-17,Not Rated,PG,PG-13,Passed,R,TV-14,TV-G,TV-PG,Unrated,X
title_year,,,,,,,,,,,,,,,
2012.0,0.0,2.0,0.0,0.0,0.0,18.0,26.0,71.0,0.0,84.0,0.0,1.0,0.0,0.0,0.0
2013.0,0.0,1.0,0.0,0.0,0.0,14.0,26.0,68.0,0.0,93.0,1.0,0.0,0.0,1.0,0.0
2014.0,0.0,3.0,0.0,0.0,0.0,9.0,28.0,72.0,0.0,86.0,0.0,0.0,0.0,5.0,0.0
2015.0,0.0,1.0,0.0,0.0,0.0,5.0,22.0,59.0,0.0,77.0,1.0,0.0,0.0,1.0,0.0
2016.0,0.0,0.0,0.0,0.0,0.0,2.0,11.0,41.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0


## Pivot table

In [83]:
df.head(3)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000


In [90]:
(df
 .pivot_table(
     values='movie_title', 
     index='title_year', 
     columns='content_rating', 
     aggfunc=pd.DataFrame.nunique)
.fillna(0)
.tail(5))

content_rating,Approved,G,GP,M,NC-17,Not Rated,PG,PG-13,Passed,R,TV-14,TV-G,TV-PG,Unrated,X
title_year,,,,,,,,,,,,,,,
2012.0,0.0,2.0,0.0,0.0,0.0,18.0,26.0,71.0,0.0,84.0,0.0,1.0,0.0,0.0,0.0
2013.0,0.0,1.0,0.0,0.0,0.0,14.0,26.0,68.0,0.0,93.0,1.0,0.0,0.0,1.0,0.0
2014.0,0.0,3.0,0.0,0.0,0.0,9.0,28.0,72.0,0.0,86.0,0.0,0.0,0.0,5.0,0.0
2015.0,0.0,1.0,0.0,0.0,0.0,5.0,22.0,59.0,0.0,77.0,1.0,0.0,0.0,1.0,0.0
2016.0,0.0,0.0,0.0,0.0,0.0,2.0,11.0,41.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0


values : column to aggregate, optional

index : column, Grouper, array, or list of the previous

    If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

columns : column, Grouper, array, or list of the previous

    If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

aggfunc : function or list of functions, default numpy.mean

    If list of functions passed, the resulting pivot table will have hierarchical columns whose top level are the function names (inferred from the function objects themselves)

fill_value : scalar, default None

    Value to replace missing values with

margins : boolean, default False

    Add all row / columns (e.g. for subtotal / grand totals)

dropna : boolean, default True

    Do not include columns whose entries are all NaN

margins_name : string, default ‘All’

    Name of the row / column that will contain the totals when margins is True.


# Plotting

In [124]:
to_plot = df.groupby(df.title_year).agg({df.duration.name:{'m':np.mean, 'sdf':np.std}})

to_plot.columns = to_plot.columns.droplevel()

#to_plot = to_plot.assign(low = lambda df: df.mean - df.std)

to_plot

/home/tommy/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,m,sdf
title_year,,
1916.0,123.000000,NaN
1920.0,110.000000,NaN
1925.0,151.000000,NaN
1927.0,145.000000,NaN
1929.0,105.000000,7.071068
1930.0,96.000000,NaN
1932.0,79.000000,NaN
1933.0,77.500000,16.263456
1934.0,65.000000,NaN
